# Agent Streaming and Responses

Master agent streaming modes and structured outputs.

**What you'll learn:**
- Agent streaming: messages, updates, values modes
- Structured output: Type-safe responses with Pydantic

#### Agent Streaming Modes

| Mode | Use Case | Returns |
|------|----------|---------|
| **messages** | Real-time token display | Individual message chunks as generated |
| **updates** | Debugging agent flow | Node name + output after each node executes |
| **values** | Track full state | Complete state snapshot after each step |

In [1]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
from scripts import base_tools

In [3]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [4]:
conn = sqlite3.connect("db/streaming_agent.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    checkpointer=checkpointer
)

In [5]:
# Stream mode: messages
for chunk in agent.stream(
    {'messages': [HumanMessage('Search for tech news')]},
    stream_mode='messages',
    config={'configurable': {'thread_id': 'stream_msg'}}
):
    print(chunk)

(AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "tech news"}'}, '__gemini_function_call_thought_signatures__': {'1ba1c25e-2e29-4452-8526-e3006da0b940': 'CiQBcsjafPrH5KGG7GRI5L2RHjYD9IyoBJHJnzhZwfagb0wEU2UKWwFyyNp8dMajIyzRSpY47a8Sh+MWwZT8qsyIvxRnWRdOV389Ln3ZntCFsX5UgmqT1pruLiw4JcOxIY47Q8VdnrPw2QafyKa1dJIgBn/9Oq1cGddO3mYJugXjlaIKlQEBcsjafMg10hioS38QtkJu1geIp4sZFJCB3nsUly++uZ8qICaydn9USjTi6nrn/V2IiQYDHqQdZO0AXc+4c5cokZuCD1yEKXG15YJg83JCM2FUL9YErcg9FV/ieLCXdBiQEHGa63nRrEfCNh7/usXWMByFI47UUUFWkfZl19jXUZRSpKtL2W07DOxXhotNbAkhSOLUsg=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc33e-c5d4-7270-9135-09470e82b339', tool_calls=[{'name': 'web_search', 'args': {'query': 'tech news'}, 'id': '1ba1c25e-2e29-4452-8526-e3006da0b940', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 86, 'output_tokens': 6

In [6]:
# Stream mode: updates
for chunk in agent.stream(
    {'messages': [HumanMessage('Search for AI news')]},
    stream_mode='updates',
    config={'configurable': {'thread_id': 'stream_upd'}}
):
    print(chunk)

{'model': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "AI news"}'}, '__gemini_function_call_thought_signatures__': {'8786109e-23bf-40bf-84dd-56bcfa5051f3': 'CssBAXLI2nw40zEluwJT8J7Fki3As7BcOKBYtaiicpciAtp2yI5AJfqOJfu1J8VgoEYk1LcY1uoj/DO4lCGNTJPkRIJT10zYpkmWT57mPzoST7On3j7MRdEfRQd1uFdgz6egG81xsK0tLS/eenUulEZgK8KRsySfEbE48PHnuucOqKAXaLxm5HKcvUJhWq2tu/j5mDRKMqHPRw+0Hq6B5Ky4LIlbh9JRuuX/oUGQHzjd5RQrCFonMPt5iDE1x1xbrvNlLVXndEuNuVKyKp8='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc33e-e904-7cf0-b84d-d6484fa49546-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'AI news'}, 'id': '8786109e-23bf-40bf-84dd-56bcfa5051f3', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 86, 'output_tokens': 61, 'total_tokens': 147, 'input_token_details': {'cache_read': 0}, 'output_token_details'

In [7]:
# Stream mode: values
for step, chunk in enumerate(agent.stream(
    {'messages': [HumanMessage('What is ML?')]},
    stream_mode='values',
    config={'configurable': {'thread_id': 'stream_val'}}
)):
    print(f"Step {step}: {len(chunk.get('messages', []))} messages")

Step 0: 1 messages
Step 1: 2 messages
Step 2: 3 messages
Step 3: 4 messages


In [8]:
from scripts.agent_utils import stream_agent_response

stream_agent_response(agent, 'Search for AI news', thread_id='stream_util')


  Tool Called: web_search
   Args: {'query': 'AI news'}


  Tool Result (length: 23778 chars)

Here's a summary of the latest AI news:

**From AI News:**
*   New ETSI standard for AI security introduced.
*   McKinsey is piloting an AI chatbot for graduate recruitment.
*   OpenAI, Google, and Anthropic are intensifying their competition in AI medical diagnostics with new tools.
*   AstraZeneca is leveraging in-house AI to accelerate oncology research.
*   Research indicates that young adults in the UK are open to using AI for financial advice.
*   There's a focus on addressing workforce anxiety related to AI integration.
*   Insights are emerging on why Apple chose Google for its AI features, providing lessons for enterprise AI buyers regarding the Gemini deal.

**From TechCrunch:**
*   AI video startup Higgsfield, co-founded by a former Snap executive, has achieved a $1.3 billion valuation.
*   The US has imposed a 25% tariff on Nvidia’s H200 AI chips destined for China.
*   OpenAI ha

## Structured Output

In [9]:
from pydantic import BaseModel, Field
from typing import Optional
from langchain.agents.structured_output import ToolStrategy, ProviderStrategy

class FinancialAnalysis(BaseModel):
    company: str = Field(description="Company name")
    stock_symbol: str = Field(description="Stock ticker")
    current_price: Optional[str] = Field(description="Current price", default=None)
    analysis: str = Field(description="Brief analysis")
    recommendation: str = Field(description="Buy/Hold/Sell")

structured_agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    response_format=ToolStrategy(FinancialAnalysis)  # ProviderStrategy
)

response = structured_agent.invoke({
    'messages': [HumanMessage('Analyze Tesla stock')]
})

response

{'messages': [HumanMessage(content='Analyze Tesla stock', additional_kwargs={}, response_metadata={}, id='da873477-c64f-4bc3-bdaf-c7ee5279d2ef'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'FinancialAnalysis', 'arguments': '{"company": "Tesla", "stock_symbol": "TSLA", "analysis": "None", "recommendation": "None"}'}, '__gemini_function_call_thought_signatures__': {'22797d9f-344f-4881-a996-85bb4e57dcd5': 'CvoEAXLI2nxthJbAukn0XD7ocBQc6Vp+n0gstyraA/cB49euqu4BsXhy7k9RKAYURrL3Yzknr7/HpNzhWP/S2dLD020OucpwfUhTjykXU+7GqKsqmyLBAhUQakMkC6cV2XuznPTKTdC1zUm89o8sMHEya3u3D+OtsdX/hdogNNIC8bk4YptVUSrSBrv39Ch/fNVUzl6cENeEYa8ERsl+jOmMrXOxd34Q72UvyUfeI2iKR1lrPKBEFm337SJxDsjaNdPj7MyztiHWsVUCfIK8HirMfcVdPdGAqhwY3eTh+GkfaLc5B+xMLx5sIDaxmBCj/huGsmUswMzz+T7E+nRnjWompgk6rgsNDvkIc7D26GvJXF45soZPFqb6r5i9Tv0aSXhSpsjtaZPWbSnmcu+j4LEKNlNHI/PJ+g+LY0hYmYa4c912XTX39s2ru+dS4ekikfNNKbOQc2XnC/tUlyn6R8ufWnCcVFSWAkrCy8Bi+/smIbtOnhKha+Zfugj18Nl0EkGFrKSYSwYlb14Mjy4BYaczMPG9oDGqxTBlVWrL93Z+8ARY58Fcmst